In [4]:
options(repos = c(CRAN = "https://cloud.r-project.org"))
library(ggplot2)
library(RobustGaSP)
install.packages("gridExtra")  # Add this package for plotting
library(gridExtra)  # Load the package after installation


data <- read.csv("droplet_data.csv", header = TRUE)  # Use header=TRUE because your first row has column names
clean_data <- na.omit(data)

x <- clean_data[, 1:3]
y <- clean_data[, 12:13]

x_scaled <- scale(x)

#########

##
## Robust Gaussian Stochastic Process, RobustGaSP Package

## Copyright (C) 2016-2025 Mengyang Gu, Jesus Palomo and James O. Berger

#########


Attaching package: ‘RobustGaSP’


The following object is masked from ‘package:stats’:

    simulate





The downloaded binary packages are in
	/var/folders/0n/73z0s4614xjfsylq8hhskhhm0000gn/T//Rtmph1YqbT/downloaded_packages


In [5]:
model <- ppgasp(
  design = x_scaled,
  response = y,
  nugget.est=TRUE, 
  num_initial_values=3
)

The upper bounds of the range parameters are 27.46639 28.34687 26.16567 Inf 
The initial values of range parameters are 0.5493278 0.5669373 0.5233133 
Start of the optimization  1  : 
The number of iterations is  23 
 The value of the  marginal posterior  function is  10130.35 
 Optimized range parameters are 4.385126 12.37087 4.225211 
 Optimized nugget parameter is 1.470256e-06 
 Convergence:  TRUE 
The initial values of range parameters are 2.406365 2.483504 2.292407 
Start of the optimization  2  : 
The number of iterations is  29 
 The value of the  marginal posterior  function is  10130.35 
 Optimized range parameters are 4.385108 12.37082 4.225183 
 Optimized nugget parameter is 1.470292e-06 
 Convergence:  FALSE 
The initial values of range parameters are 0.003007024 0.0006458113 0.001250513 
Start of the optimization  3  : 
The number of iterations is  51 
 The value of the  marginal posterior  function is  10130.35 
 Optimized range parameters are 4.385128 12.37086 4.225211 


In [ ]:
library(akima)

parameters_given_mfr <- function(mfr_threshold, n_simulations = 1000) {
  # Generate random inputs
  G <- runif(n_simulations, min = 10, max = 30)
  wave <- runif(n_simulations, min = 1/300, max = 1/100)
  Temp <- runif(n_simulations, min = 66, max = 270)

  ensembles <- data.frame(G = G, wave = wave, Temp = Temp)

  # Normalize
  G_mean <- mean(G); G_std <- sd(G)
  wave_mean <- mean(wave); wave_std <- sd(wave)
  Temp_mean <- mean(Temp); Temp_std <- sd(Temp)

  ensembles_scaled <- data.frame(
    G = (G - G_mean) / G_std,
    wave = (wave - wave_mean) / wave_std,
    Temp = (Temp - Temp_mean) / Temp_std
  )

  # Predict
  model.pred <- predict(model, ensembles_scaled)
  mfr_values <- model.pred$mean[,1] / model.pred$mean[,2]

  # Filter
  passed <- ensembles[mfr_values > mfr_threshold, ]
  passed_mfr <- mfr_values[mfr_values > mfr_threshold]

  if (nrow(passed) < 10) {
    warning("Not enough points passed the threshold to interpolate.")
    return(NULL)
  }

  # Interpolate using akima
  interp_result <- akima::interp(
    x = passed$G,
    y = passed$Temp,
    z = passed_mfr,
    xo = seq(min(passed$G), max(passed$G), length = 100),
    yo = seq(min(passed$Temp), max(passed$Temp), length = 100),
    duplicate = "mean"
  )

  # Plot
  filename <- sprintf("parameter_distribution_mfr_%.7f.pdf", mfr_threshold)
  pdf(filename, width = 10, height = 8)
  filled.contour(
    x = interp_result$x,
    y = interp_result$y,
    z = interp_result$z,
    xlab = "G (kg/m²s)",
    ylab = "Temperature (K)",
    main = sprintf("MFR Surface for MFR > %.7f", mfr_threshold),
    color.palette = colorRampPalette(c("blue", "white", "red")),
    key.title = title(main = "MFR", cex.main = 1),
    key.axes = axis(4, cex.axis = 0.7)
  )
  dev.off()
}

In [ ]:
parameters_given_mfr(0.000003, n_simulations = 100000)

pdf 
  2

In [ ]:
library(sensitivity)

# --- Step 1: Assume you already have a trained GP model called `model`
# Make sure it’s trained and has attributes for scaling (center and scale)

# --- Step 2: Generate Sobol samples in unit cube ---
n <- 10000
X1_raw <- matrix(runif(n * 3), ncol = 3)
X2_raw <- matrix(runif(n * 3), ncol = 3)

# --- Step 3: Scale inputs to match training GP space ---
scale_to_training <- function(new_x, train_x_scaled) {
  center <- attr(train_x_scaled, "scaled:center")
  scale <- attr(train_x_scaled, "scaled:scale")
  sweep(sweep(new_x, 2, center, "-"), 2, scale, "/")
}

# Assume `train_x_scaled` is the scaled training input matrix used to train the GP
X1 <- scale_to_training(X1_raw, x_scaled)
X2 <- scale_to_training(X2_raw, x_scaled)

# --- Step 4: Build Sobol model structure ---
sobol_model <- sobol2007(model = NULL, X1 = X1, X2 = X2, nboot = 0)

# --- Step 5: Predict using RobustGaSP or another GP ---
# Ensure `model` is already defined
pred <- predict(model, testing_input = sobol_model$X)$mean  # Nx2 matrix

# --- Step 6: Perform Sobol sensitivity analysis for both outputs ---
sobol_model_y1 <- tell(sobol_model, pred[, 1])
sobol_model_y2 <- tell(sobol_model, pred[, 2])

# --- Step 7: Output results ---
cat("First-order Sobol indices for y1:\n")
print(sobol_model_y1$S)

cat("\nFirst-order Sobol indices for y2:\n")
print(sobol_model_y2$S)

ERROR: Error in `.rowNamesDF<-`(x, value = value): invalid 'row.names' length


In [ ]:
pdf_given_mfr <- function(mfr_threshold, model, n_simulations = 1000) {
  df_list <- list()

  for (i in 1:n_simulations) {
    G_batch <- rep(runif(1, 10, 30), 1000)
    wave <- runif(1000, min = 1/300, max = 1/100)
    Temp_batch <- rep(runif(1, 66, 270), 1000)

    # Normalize
    G_mean <- 19.99292310090832; G_std <- 5.768017604154681
    wave_mean <- mean(wave); wave_std <- sd(wave)
    Temp_mean <- 168.00554590178015; Temp_std <- 58.87173432703981

    if (wave_std == 0) next  # prevent divide-by-zero

    ensembles_scaled <- data.frame(
      G = (G_batch - G_mean) / G_std,
      wave = (wave - wave_mean) / wave_std,
      Temp = (Temp_batch - Temp_mean) / Temp_std
    )

    # Predict
    model.pred <- predict(model, ensembles_scaled)
    mfr <- sum(model.pred$mean[, 1]) / sum(model.pred$mean[, 2])
    # Filter
    if (mfr >= mfr_threshold) {
      df_list[[length(df_list) + 1]] <- data.frame(
        volume = model.pred$mean[, 1],
        time = model.pred$mean[, 2]
      )
    }
  }

  df <- do.call(rbind, df_list)

  # Plotting
  library(ggplot2)
  library(gridExtra)

  p1 <- ggplot(df, aes(x = volume)) +
    geom_histogram(aes(y = ..density..), bins = 100, fill = "skyblue", color = "black") +
    theme_minimal() +
    labs(title = "PDF of Volume", x = "Volume", y = "Density")

  p2 <- ggplot(df, aes(x = time)) +
    geom_histogram(aes(y = ..density..), bins = 100, fill = "salmon", color = "black") +
    theme_minimal() +
    labs(title = "PDF of Time", x = "Time", y = "Density")

  filename <- sprintf("pdf_mfr_%.7f.pdf", mfr_threshold)
  pdf(filename, width = 12, height = 6)
  grid.arrange(p1, p2, ncol = 2)
  dev.off()

  return(df)
}

In [32]:
pdf_given_mfr(0.000003, model, n_simulations = 1000)

volume,time
<dbl>,<dbl>
3.728868e-09,0.002688428
2.248472e-08,0.006055112
3.933646e-09,0.002750717
3.092041e-09,0.002489154
1.193132e-08,0.004506060
1.949544e-08,0.005645490
2.321648e-08,0.006154576
2.543342e-09,0.002307680
8.956674e-09,0.003959044
